Mounting Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord

# **PARSING**

Traversing The MIDI files and Parsing It

In [ ]:
notes = []
try:
  for file in glob.glob("/content/drive/My Drive/Music Creation/Database/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    try:
      s2 = instrument.partitionByInstrument(midi)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/drive/My Drive/Music Creation/Data/notes', 'wb') as filepath:
      pickle.dump(notes, filepath)
except exception as e :
  print(e)


# **ENCODING**

Mapping function to map from string-based categorical data to integer-based numerical data

In [6]:
n_vocab = len(set(notes))
sequence_length = 100 # can be changed because this is the number which is going to go as a input in RNN 
sorted_notes = sorted(set(item for item in notes))
map_notes = dict((note, number) for number, note in enumerate(sorted_notes))

X=[]
y=[]

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    X.append([map_notes[char] for char in sequence_in])
    y.append(map_notes[sequence_out])

n_patterns = len(X)
X = np.reshape(X, (n_patterns, sequence_length, 1))
X = X / float(n_vocab)
from keras.utils import np_utils
y= np_utils.to_categorical(y)

# **MODEL**

*  LSTM layers
*  Dropout layers
*  Dense layers
*  Activation layer





In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [ ]:
filepath = "/content/drive/My Drive/Music Creation/Weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=0,save_best_only=True,mode='min')
model.fit(X, y, epochs=100, batch_size=128,callbacks=[checkpoint])

Epoch 1/100
446/446 [==============================] - 82s 185ms/step - loss: 4.7536
Epoch 2/100
446/446 [==============================] - 82s 185ms/step - loss: 4.5926
Epoch 3/100
446/446 [==============================] - 82s 184ms/step - loss: 4.6006
Epoch 4/100
446/446 [==============================] - 82s 184ms/step - loss: 4.5307
Epoch 5/100
446/446 [==============================] - 82s 184ms/step - loss: 4.4542
Epoch 6/100
446/446 [==============================] - 82s 184ms/step - loss: 4.3849
Epoch 7/100
446/446 [==============================] - 82s 184ms/step - loss: 4.2909
Epoch 8/100
446/446 [==============================] - 82s 185ms/step - loss: 4.1863
Epoch 9/100
446/446 [==============================] - 82s 184ms/step - loss: 4.0616
Epoch 10/100
446/446 [==============================] - 82s 184ms/step - loss: 3.9135
Epoch 11/100
446/446 [==============================] - 82s 184ms/step - loss: 3.7487
Epoch 12/100
446/446 [==============================] - 82s 184

This is just how to get a output using the model

In [4]:
with open('/content/drive/My Drive/Music Creation/Data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

n_vocab = len(set(notes))
sorted_notes = sorted(set(item for item in notes))
map_notes = dict((note, number) for number, note in enumerate(sorted_notes))

sequence_length = 100
X = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    X.append([map_notes[char] for char in sequence_in])


n_patterns = len(X)
X_normalized = np.reshape(X, (n_patterns, sequence_length, 1))
X_normalized = X_normalized / float(n_vocab)


In [5]:
model = Sequential()
model.add(LSTM(256,input_shape=(X_normalized.shape[1], X_normalized.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.load_weights('/content/drive/My Drive/Music Creation/Weights/weights.hdf5')

In [6]:
from music21 import stream
start = np.random.randint(0, len(X)-1)
int_to_note = dict((number, note) for number, note in enumerate(sorted_notes))
pattern = X[start]
prediction_output = []

for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_sample.mid')

'test_sample.mid'